In [1]:
%load_ext autoreload
%autoreload 2

Pytorch EO core design revolves around `Tasks`. Examples of tasks are: image classification, object detection, image segmentation, etc.

A task is defined by:
1. `model`: a neural network that will be trained to perform the task
2. `hparams`: hyperparameters used to train the model
3. `metrics`: evaluation of the model in the task

## Models

Models are neural networks that take inputs and produce outputs. You can build your own or use third party models. By default we use `torchvision`, so you can use any model from there

In [2]:
import torch 
from pytorch_eo.tasks.BaseTask import BaseTask

task = BaseTask('resnet18')

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 1000])

In case you use models from torchvision, you can pass any parameter in the `hparams` object as follows.

In [3]:
hparams = {
    'model_params': {
        'pretrained': True
    }
}

task = BaseTask('resnet18', hparams)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 1000])

You can always make your own models, for example modifying an existing model from torchvision.

In [4]:
import torch
import torchvision 

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 10)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 10])

Or building your own model from scratch.

In [5]:
class MyModel(torch.nn.Module):
    def __init__(self, pretrained, num_classes):
        super().__init__()
        resnet = torchvision.models.resnet18(pretrained=pretrained)
        self.backbone = torch.nn.Sequential(*list(resnet.children())[:-2])
        self.fc = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d((1, 1)),
            torch.nn.Flatten(),
            torch.nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        f = self.backbone(x)
        return self.fc(f)

In [6]:
model = MyModel(pretrained=True, num_classes=10)

task = BaseTask(model)

output = task(torch.randn(32, 3, 224,224))
output.shape

torch.Size([32, 10])

But you would probably want to use third party implementations with extra functionality.

In [7]:
import timm 

model = timm.create_model(
    'resnet18',
    pretrained='imagenet',
    in_chans=3,
    num_classes=10,
    features_only=True
)


task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
for o in output:
    print(o.shape)

torch.Size([32, 64, 112, 112])
torch.Size([32, 64, 56, 56])
torch.Size([32, 128, 28, 28])
torch.Size([32, 256, 14, 14])
torch.Size([32, 512, 7, 7])


In [8]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name='resnet18',
    encoder_weights='imagenet',
    in_channels=3,
    classes=2,
)

task = BaseTask(model)

output = task(torch.randn(32,3,224,224))
output.shape

torch.Size([32, 2, 224, 224])

## hparams

This is a `dict` holding the hyperparameters used for training. Pytorch lightning will save the object in the model's checkpoint (that can be used to resume training, for example) so make sure to add any additional information that you want to save. In some cases, default parameters will be used if hparams are not provided.

In [9]:
hparams = {
    # hparams used for training
    'model_params': { # only if you use default torchvision models
        'pretrained': True
    },
    'loss': 'CrossEntropyLoss', # choose one from pytorch docs
    'loss_params': {}, 
    'optimizer': 'Adam', # choose one from pytorch docs
    'optim_params': {
        'lr': 1e-4
    },
    'scheduler': 'CosineAnnealingLR', # choose one from pytorch docs
    'scheduler_params': {
        'T_max': 10,
        'verbose': True
    }
    # extra
    # epochs, batch size, model, transforms, ...
}

## Metrics

The last piece to train a model is the metrics. You can train without metrics, use our simple convenient implementations, or use your own implementations. In most tasks we will use at least one metric even if they are not provided (they are that important !)

In [10]:
import pytorch_lightning as pl

from pytorch_eo.datasets.eurosat import EuroSATRGB
from pytorch_eo.tasks.classification import ImageClassification
from pytorch_eo.metrics.classification import accuracy

ds = EuroSATRGB(batch_size=32)

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, ds.num_classes)

hparams = {
	'loss': 'CrossEntropyLoss',
	'optimizer': 'Adam'
}

metrics = {'acc': accuracy}

task = ImageClassification(model, hparams, metrics)

In [11]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


/home/juan/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/juan/miniconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/juan/miniconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  warning_cache.deprecation(


You can use as many metrics as you want (all will be logged)

In [12]:
def my_accuracy(y_hat, y):
    return (torch.argmax(y_hat, axis=1) == y).sum() / y.shape[0]

metrics = {'acc': accuracy, 'my_acc': my_accuracy}

task = ImageClassification(model, hparams, metrics)

In [13]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


But you would probably want to use a third party library like `torchmetrics` with a lot of tested implementations.

In [14]:
from torchmetrics.functional import accuracy as tm_accuracy
from torchmetrics import Accuracy # this is broken

# torchmetrics wants probas
def my_tm_accuracy(y_pred, y):
    y_pred = torch.softmax(y_pred, 1)
    return tm_accuracy(y_pred, y)

metrics = {'acc': accuracy, 'my_acc': my_accuracy, 'tm_acc': my_tm_accuracy}

task = ImageClassification(model, hparams, metrics)

In [15]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=3,
    limit_train_batches=10
)

trainer.fit(task, ds)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)
